In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

# ============================
# 1. Data Loading
# ============================
df_assessments = pd.read_csv("assessments.csv")
df_courses = pd.read_csv("courses.csv")
df_studentAssessment = pd.read_csv("studentAssessment.csv")
df_studentInfo = pd.read_csv("studentInfo.csv")
df_studentRegistration = pd.read_csv("studentRegistration.csv")
df_studentVle = pd.read_csv("studentVle.csv")
df_vle = pd.read_csv("vle.csv")

# ============================
# 2. Merge studentAssessment & studentInfo
#    to get code_module & code_presentation
# ============================
# Note: A single student can appear in multiple modules/presentations.
# This merge will replicate rows if a student is in multiple modules.
# Adjust your data logic if needed for multi-module students.
df_studentAssessment_expanded = pd.merge(
    df_studentAssessment,
    df_studentInfo[["id_student", "code_module", "code_presentation"]],
    on="id_student",
    how="left"
)

# Now merge with assessments to get 'assessment_type'
df_studentAssessment_expanded = pd.merge(
    df_studentAssessment_expanded,
    df_assessments[["id_assessment", "assessment_type", "code_module", "code_presentation"]],
    on=["id_assessment", "code_module", "code_presentation"],
    how="left"
)

# ============================
# 3. Create 2 Assessment Features
#    (Avg_TMA_Score & Exam_Score)
# ============================
# Filter TMA vs. Exam
df_avg_tma = (
    df_studentAssessment_expanded[df_studentAssessment_expanded["assessment_type"] == "TMA"]
    .groupby(["id_student", "code_module", "code_presentation"])["score"]
    .mean()
    .reset_index()
    .rename(columns={"score": "Avg_TMA_Score"})
)

df_exam_score = (
    df_studentAssessment_expanded[df_studentAssessment_expanded["assessment_type"] == "Exam"]
    .groupby(["id_student", "code_module", "code_presentation"])["score"]
    .mean()
    .reset_index()
    .rename(columns={"score": "Exam_Score"})
)

# Merge TMA & Exam into a single assessment-features table
df_assess_feats = pd.merge(
    df_avg_tma,
    df_exam_score,
    on=["id_student", "code_module", "code_presentation"],
    how="outer"
).fillna(0)

# ============================
# 4. Create 40 VLE Features
#    (20 sum of clicks + 20 visits)
# ============================
# Merge studentVle with vle to get activity_type
df_studentVle_expanded = pd.merge(
    df_studentVle,
    df_vle[["id_site", "activity_type", "code_module", "code_presentation"]],
    on=["id_site", "code_module", "code_presentation"],
    how="left"
)

# (a) sum_clicks per (id_student, code_module, code_presentation, activity_type)
pivot_sum = (
    df_studentVle_expanded
    .groupby(["id_student", "code_module", "code_presentation", "activity_type"])["sum_click"]
    .sum()
    .unstack("activity_type", fill_value=0)
)
pivot_sum.columns = [f"sum_clicks_{col}" for col in pivot_sum.columns]
pivot_sum.reset_index(inplace=True)

# (b) number_of_visits per activity_type
#    define a "visit" as any row with sum_click > 0
df_studentVle_expanded["visit"] = df_studentVle_expanded["sum_click"].apply(lambda x: 1 if x > 0 else 0)
pivot_visits = (
    df_studentVle_expanded
    .groupby(["id_student", "code_module", "code_presentation", "activity_type"])["visit"]
    .sum()
    .unstack("activity_type", fill_value=0)
)
pivot_visits.columns = [f"visits_{col}" for col in pivot_visits.columns]
pivot_visits.reset_index(inplace=True)

# Merge sum_clicks & visits into one DF
df_vle_feats = pd.merge(
    pivot_sum, 
    pivot_visits, 
    on=["id_student", "code_module", "code_presentation"],
    how="outer"
).fillna(0)

# ============================
# 5. Merge Demographics + Assessments + VLE
# ============================
# We want 8 demographic columns from studentInfo:
#   1) gender
#   2) region
#   3) highest_education
#   4) imd_band
#   5) age_band
#   6) disability
#   7) studied_credits
#   8) num_of_prev_attempts
# Also keep final_result for target creation.

df_merged = pd.merge(
    df_studentInfo,
    df_assess_feats,    # (Avg_TMA_Score, Exam_Score)
    on=["id_student", "code_module", "code_presentation"],
    how="left"
)

df_merged = pd.merge(
    df_merged,
    df_vle_feats,       # 40 VLE columns
    on=["id_student", "code_module", "code_presentation"],
    how="left"
).fillna(0)

# ============================
# 6. Remove Withdrawn & Create Binary Target
# ============================
df_merged = df_merged[df_merged["final_result"] != "Withdrawn"].copy()

df_merged["target"] = df_merged["final_result"].apply(
    lambda x: 1 if x in ["Pass", "Distinction"] else 0
)

# ============================
# 7. Select Final Columns (50 features + target)
# ============================
# 8 Demographic Features
demographic_cols = [
    "gender", "region", "highest_education", "imd_band",
    "age_band", "disability", "studied_credits", "num_of_prev_attempts"
]

# 2 Assessment Features
assessment_cols = ["Avg_TMA_Score", "Exam_Score"]

# 40 VLE Features = all pivot columns except the merge keys
all_vle_cols = set(df_vle_feats.columns) - {"id_student", "code_module", "code_presentation"}
vle_cols = list(all_vle_cols)  # convert set to list to preserve them

# Final feature list
feature_cols = demographic_cols + assessment_cols + vle_cols

# Keep IDs if you want them for reference
id_cols = ["id_student", "code_module", "code_presentation"]

# Build final DataFrame
df_final = df_merged[id_cols + feature_cols + ["target"]].copy()

print(df_final.head(10))
print("Number of rows in df_final:", len(df_final))
print("Number of features (excluding target):", len(feature_cols))

# Rename columns for clarity
df_final.rename(
    columns={
        "highest_education": "HLE",
        "age_band": "Age group",
        "studied_credits": "Credit Distribution",
        "gender": "Gender",
        "region": "Region"
    },
    inplace=True
)



    id_student code_module code_presentation gender                region  \
0        11391         AAA             2013J      M   East Anglian Region   
1        28400         AAA             2013J      F              Scotland   
3        31604         AAA             2013J      F     South East Region   
4        32885         AAA             2013J      F  West Midlands Region   
5        38053         AAA             2013J      M                 Wales   
6        45462         AAA             2013J      M              Scotland   
7        45642         AAA             2013J      F  North Western Region   
8        52130         AAA             2013J      F   East Anglian Region   
9        53025         AAA             2013J      M          North Region   
10       57506         AAA             2013J      M          South Region   

              highest_education imd_band age_band disability  studied_credits  \
0              HE Qualification  90-100%     55<=          N           

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

# ===========================
# Data Preprocessing
# ===========================

# Load your processed dataframe
df = df_final.copy()

# Drop student ID (not relevant for clustering)
df = df.drop(columns=['id_student'])

print(df.head())

# Encode categorical variables
categorical_cols = ['Gender', 'Region', 'HLE', 'Age group', 'code_module']
label_encoders = {col: LabelEncoder() for col in categorical_cols}

for col in categorical_cols:
    df[col] = label_encoders[col].fit_transform(df[col])

# Standardize features for clustering
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

# ===========================
# Finding Optimal K (Elbow & Silhouette Method)
# ===========================

wcss = []  # Within-cluster sum of squares
silhouette_scores = []

K_range = range(2, 31)  # Trying K from 2 to 30
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(df_scaled)
    wcss.append(kmeans.inertia_)  # Inertia = WCSS
    silhouette_scores.append(silhouette_score(df_scaled, cluster_labels))

# Plot Elbow Method
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(K_range, wcss, marker='o')
plt.xlabel("Number of Clusters (K)")
plt.ylabel("WCSS (Within-cluster Sum of Squares)")
plt.title("Elbow Method")

# Plot Silhouette Scores
plt.subplot(1, 2, 2)
plt.plot(K_range, silhouette_scores, marker='o', color='r')
plt.xlabel("Number of Clusters (K)")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Score Method")

plt.tight_layout()
plt.show()

# ===========================
# Applying K-Means with Optimal K
# ===========================

optimal_k = 10  # Change this based on elbow method & silhouette score analysis
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
df['Cluster'] = kmeans.fit_predict(df_scaled)

# ===========================
# Visualizing Clusters using PCA (Better for 20 Clusters)
# ===========================

pca = PCA(n_components=2)
df_pca = pca.fit_transform(df_scaled)
df['PCA1'] = df_pca[:, 0]
df['PCA2'] = df_pca[:, 1]

plt.figure(figsize=(10, 7))

# Use a more distinct colormap
scatter = sns.scatterplot(
    x=df['PCA1'], y=df['PCA2'], hue=df['Cluster'], 
    palette="tab20", alpha=0.7, edgecolor="black"
)

plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.title(f"K-Means Clustering (K={optimal_k}) - PCA Projection")

# Adjust legend for readability
legend = scatter.legend_
for t in legend.texts:
    t.set_fontsize(9)  # Reduce font size for better visibility

plt.show()

# ===========================
# Display Cluster Statistics
# ===========================

print(df.groupby('Cluster').mean())  # Analyze the characteristics of each cluster


  code_module code_presentation Gender                Region  \
0         AAA             2013J      M   East Anglian Region   
1         AAA             2013J      F              Scotland   
3         AAA             2013J      F     South East Region   
4         AAA             2013J      F  West Midlands Region   
5         AAA             2013J      M                 Wales   

                     HLE imd_band Age group disability  Credit Distribution  \
0       HE Qualification  90-100%      55<=          N                  240   
1       HE Qualification   20-30%     35-55          N                   60   
3  A Level or Equivalent   50-60%     35-55          N                   60   
4     Lower Than A Level   50-60%      0-35          N                   60   
5  A Level or Equivalent   80-90%     35-55          N                   60   

   num_of_prev_attempts  ...  visits_url  sum_clicks_questionnaire  \
0                     0  ...         4.0                       0.0   


ValueError: could not convert string to float: '2013J'